In [ ]:
import ase.io.lammpsdata as io
from ase.visualize import view#
import os
import numpy as np
import ase.io as ase
cell_size = 14.1328

tol = cell_size/8.0
cell = cell_size/(8.0)


In [ ]:
def cell_cutter(initial,final,stepafter):

    atoms = io.read_lammps_data(initial,style="atomic")
    nextatoms = io.read_lammps_data(final,style="atomic")
    afteratoms = io.read_lammps_data(stepafter,style="atomic")

    diff = atoms.get_positions()-nextatoms.get_positions()
    store = 0
    entry = []
    for i in diff:
        current = np.linalg.norm(i)
        if current > 1.0:
            store = i
            entry.append(i)
            
    atom_pos =atoms.get_positions()[np.where(store==diff)[0][0]]
    afteratom_pos =afteratoms.get_positions()[np.where(store==diff)[0][0]]


    return  len(entry),nextatoms.get_positions()[np.where(store==diff)[0][0]],atom_pos,afteratom_pos
    


In [ ]:
noSteps = 500

vac_pos = np.zeros((noSteps,3))
pos = np.zeros((noSteps,3))
pos_after = np.zeros((noSteps,3))
no_atoms = []
for i in range(noSteps-2):
    initial = "steps/step{}.txt".format(i)
    final = "steps/step{}.txt".format(i+1)
    after = "steps/step{}.txt".format(i+2)
    res = cell_cutter(initial,final,after)
    vac_pos[i,:] = res[1]
    no_atoms.append(res[0])
    pos[i,:] = res[2]
    pos_after[i,:] = res[3]
    

In [ ]:
# Code identifies end of flickers, is broken by flickers that are a cycle >2

prev = np.array([-1,-1,-1])
minustwo = np.array([-1,-1,-1])
minus3 = np.array([-1,-1,-1])
flickers = []
index = 0
p=0
for i in vac_pos:

    arr = [index,index-1,index-2,index-3]
    if np.all(np.abs(prev-minus3)<tol) and np.any(np.abs(minustwo-i)>tol) and \
    not(np.any(np.isin(np.where(np.array(no_atoms)>1)[0],arr))) and \
    index!=4 and index!=0 \
    and np.all(pos[index-3]-pos_after[index-3]<tol):
        
        
        print(minus3,minustwo,prev,np.around(i,decimals=1),index,index-1,index-2,index-3)
        flickers.append([index,index-1,index-2,index-3])
    index +=1
    minus3=minustwo
    minustwo = prev
    prev = np.around(i,decimals=1)
